In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.manifold import TSNE
from umap import UMAP
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import Image
import torch.nn.functional as F
import random
from torch.utils.data import DataLoader
import csv
from torchvision.models import resnet18
import matplotlib.pyplot as plt
from MulticoreTSNE import MulticoreTSNE as TSNE
from sklearn.decomposition import PCA


/home/lorma/RL_online_transfer/env/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lorma/RL_online_transfer/env/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lorma/RL_online_transfer/env/lib/python3.8/site-packages/umap/dist

# Test set

## Analyse 12 classes

In [2]:
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import Image
import torch.nn.functional as F
import random
from torch.utils.data import DataLoader, Dataset, random_split
import csv
from torchvision.models import resnet18
class CustomDataset(torch.utils.data.Dataset):

    """
    Class which create a custom dataset
    """

    def __init__(self, image_paths, labels, transform=None):
        """
        Dataset creation

        Parameters:
        --------------------------------
        images_paths : str
            paths for each image
        labels : int
            label for each images,
        transform : torchvision.transforms.Compose
            transformation applied to the tensors
        """
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        label = (
            F.one_hot(torch.tensor(label), num_classes=2).squeeze().to(torch.float32)
        )
        return image, label


class ConvertToRGB(object):
    """
    Class which convert images in 3 canal (Red, Green, Blue)
    """

    def __call__(self, img):
        """
        Convert images in RGB

        Returns :
        -------------
        img : tensor
            images with 3 canals
        """
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img

transformation = transforms.Compose(
    [
        ConvertToRGB(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
        #transforms.Lambda(lambda x: torch.flatten(x))
    ]
)


with open(os.path.join('.', "test/image_list.txt"), "r") as f:
    lines = f.readlines()

# Créer des listes pour stocker les chemins d'image et les étiquettes
image_paths_test = []
labels_test = []

# Parcourir chaque ligne du fichier texte et extraire les informations
for line in lines:
    line = line.strip().split(" ")
    if line[1] == "0" or line[1] == "1": #Commentaire ici pour avoir les 12
        image_paths_test.append("test/" + line[0])
        labels_test.append(int(line[1]))
df_test = CustomDataset(image_paths_test, labels_test, transform=transformation)
# DataLoader pour le CustomDataset
dataloader = DataLoader(df_test, batch_size=1024, shuffle=True)
#for images, labels in dataloader:
#    images_np = images.cpu().numpy()  # Les images sont déjà aplaties grâce à la transformation
#   labels_np = labels.cpu().numpy()

### PCA avant

In [ ]:
from sklearn.decomposition import PCA

# Effectuer une réduction de dimensionnalité en utilisant PCA pour avoir 50 composantes
pca_50 = PCA(n_components=50)
data_pca_50 = pca_50.fit_transform(images_np)

# Appliquer t-SNE sur les données réduites
tsne = TSNE(n_components=2, random_state=42)
data_tsne = tsne.fit_transform(data_pca_50)

# Appliquer UMAP sur les données réduites
umap_model = UMAP(n_components=2, random_state=42)
data_umap = umap_model.fit_transform(data_pca_50)

# Créer des DataFrame avec les données réduites
tsne_df = pd.DataFrame(data_tsne)
umap_df = pd.DataFrame(data_umap)

# Afficher les données réduites avec t-SNE
plt.figure(figsize=(15, 15))
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    plt.scatter(data_tsne[mask, 0], data_tsne[mask, 1], label=f'Class {i}')
plt.legend()
plt.xlabel('t-SNE component 1 after PCA')
plt.ylabel('t-SNE component 2 after PCA')
plt.show()

# Afficher les données réduites avec UMAP
plt.figure(figsize=(15, 15))
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    plt.scatter(data_umap[mask, 0], data_umap[mask, 1], label=f'Class {i}')
plt.legend()
plt.xlabel('UMAP component 1 after PCA')
plt.ylabel('UMAP component 2 after PCA')
plt.show()


In [ ]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE

# Effectuer une réduction de dimensionnalité en utilisant PCA pour avoir 50 composantes
pca_50 = PCA(n_components=50)
data_pca_50 = pca_50.fit_transform(images_np)

# Appliquer t-SNE en 3D sur les données réduites
tsne = TSNE(n_components=3, random_state=42)
data_tsne = tsne.fit_transform(data_pca_50)

# Appliquer UMAP en 3D sur les données réduites
umap_model = UMAP(n_components=3, random_state=42)
data_umap = umap_model.fit_transform(data_pca_50)

# Afficher les données réduites avec t-SNE en 3D
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    ax.scatter(data_tsne[mask, 0], data_tsne[mask, 1], data_tsne[mask, 2], label=f'Class {i}')
ax.legend()
ax.set_xlabel('t-SNE component 1 after PCA')
ax.set_ylabel('t-SNE component 2 after PCA')
ax.set_zlabel('t-SNE component 3 after PCA')
plt.show()

# Afficher les données réduites avec UMAP en 3D
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    ax.scatter(data_umap[mask, 0], data_umap[mask, 1], data_umap[mask, 2], label=f'Class {i}')
ax.legend()
ax.set_xlabel('UMAP component 1 after PCA')
ax.set_ylabel('UMAP component 2 after PCA')
ax.set_zlabel('UMAP component 3 after PCA')
plt.show()


### Pas de PCA

In [ ]:
# Perform dimensionality reduction using UMAP
umap = UMAP(n_components=2, random_state=42)
data_umap = umap.fit_transform(images_np)
image = ['aeroplane', 'bicycle', 'bus', 'car', 'horse', 
              'knife','motorcycle', 'person', 'plant', 'skateboard','train','truck']
plt.figure(figsize=(10, 10))
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    plt.scatter(data_umap[mask, 0], data_umap[mask, 1], label=f'{image[i]}', s=10)
plt.legend()
plt.xlabel('UMAP component 1')
plt.ylabel('UMAP component 2')
plt.show()

In [ ]:
from sklearn.manifold import TSNE

# Effectuer une réduction de dimensionnalité en utilisant UMAP avec 3 composantes
umap = UMAP(n_components=3, random_state=42)
data_umap = umap.fit_transform(images_np)

# Crée un DataFrame avec les données réduites
umap_df = pd.DataFrame(data_umap, columns=['Component 1', 'Component 2', 'Component 3'])

# Trace les données réduites en utilisant un graphique 3D
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 1.
    mask = labels_np == i
    ax.scatter(umap_df['Component 1'][mask], umap_df['Component 2'][mask], umap_df['Component 3'][mask], label=f'{image[i]}', s = 10)

ax.legend()
ax.set_xlabel('UMAP component 1')
ax.set_ylabel('UMAP component 2')
ax.set_zlabel('UMAP component 3')
plt.show()


# Train set

## Analyse sur 12 classes

In [ ]:
with open(os.path.join('.', "train/image_list.txt"), "r") as f:
    lines = f.readlines()  # [next(f) for _ in range(1000)]

    # Créer des listes pour stocker les chemins d'image et les étiquettes
    image_paths = []
    labels = []

    # Parcourir chaque ligne du fichier texte et extraire les informations
    for line in lines:
        line = line.strip().split(" ")
        image_paths.append("train/" + line[0])

        labels.append(int(line[1]))

    # Appliquer les transformations d'image si nécessaire
    transform_train = transforms.Compose(
        [
            ConvertToRGB(),
            transforms.Resize(256),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            transforms.Lambda(lambda x: torch.flatten(x))
        ]
    )

df_train = CustomDataset(image_paths, labels, transform=transform_train)
# DataLoader pour le CustomDataset
dataloader = DataLoader(df_train, batch_size=1024, shuffle=True)
for images, labels in dataloader:
    images_np = images.cpu().numpy()  # Les images sont déjà aplaties grâce à la transformation
    labels_np = labels.cpu().numpy()

### PCA avant

In [ ]:
from sklearn.decomposition import PCA

# Effectuer une réduction de dimensionnalité en utilisant PCA pour avoir 50 composantes
pca_50 = PCA(n_components=50)
data_pca_50 = pca_50.fit_transform(images_np)

# Appliquer t-SNE sur les données réduites
tsne = TSNE(n_components=2, random_state=42)
data_tsne = tsne.fit_transform(data_pca_50)

# Appliquer UMAP sur les données réduites
umap_model = UMAP(n_components=2, random_state=42)
data_umap = umap_model.fit_transform(data_pca_50)

# Créer des DataFrame avec les données réduites
tsne_df = pd.DataFrame(data_tsne)
umap_df = pd.DataFrame(data_umap)

# Afficher les données réduites avec t-SNE
plt.figure(figsize=(15, 15))
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    plt.scatter(data_tsne[mask, 0], data_tsne[mask, 1], label=f'Class {i}')
plt.legend()
plt.xlabel('t-SNE component 1 after PCA')
plt.ylabel('t-SNE component 2 after PCA')
plt.show()

# Afficher les données réduites avec UMAP
plt.figure(figsize=(15, 15))
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    plt.scatter(data_umap[mask, 0], data_umap[mask, 1], label=f'Class {i}')
plt.legend()
plt.xlabel('UMAP component 1 after PCA')
plt.ylabel('UMAP component 2 after PCA')
plt.show()


In [ ]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE

# Effectuer une réduction de dimensionnalité en utilisant PCA pour avoir 50 composantes
pca_50 = PCA(n_components=50)
data_pca_50 = pca_50.fit_transform(images_np)

# Appliquer t-SNE en 3D sur les données réduites
tsne = TSNE(n_components=3, random_state=42)
data_tsne = tsne.fit_transform(data_pca_50)

# Appliquer UMAP en 3D sur les données réduites
umap_model = UMAP(n_components=3, random_state=42)
data_umap = umap_model.fit_transform(data_pca_50)

# Afficher les données réduites avec t-SNE en 3D
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    ax.scatter(data_tsne[mask, 0], data_tsne[mask, 1], data_tsne[mask, 2], label=f'Class {i}', s=5)
ax.legend()
ax.set_xlabel('t-SNE component 1 after PCA')
ax.set_ylabel('t-SNE component 2 after PCA')
ax.set_zlabel('t-SNE component 3 after PCA')
plt.show()

# Afficher les données réduites avec UMAP en 3D
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    ax.scatter(data_umap[mask, 0], data_umap[mask, 1], data_umap[mask, 2], label=f'Class {i}',s = 5)
ax.legend()
ax.set_xlabel('UMAP component 1 after PCA')
ax.set_ylabel('UMAP component 2 after PCA')
ax.set_zlabel('UMAP component 3 after PCA')
plt.show()

### Pas de PCA

In [ ]:
# Perform dimensionality reduction using UMAP
umap = UMAP(n_components=2, random_state=42)
data_umap = umap.fit_transform(images_np)

# Create a DataFrame with the reduced data
umap_df = pd.DataFrame(data_umap)

# Plot the reduced data using pandas plot
plt.figure(figsize=(10, 10))
for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 11.
    mask = labels_np == i
    plt.scatter(data_umap[mask, 0], data_umap[mask, 1], label=f' {image[i]}', s = 10)
plt.legend()
plt.xlabel('t-SNE component 1')
plt.ylabel('t-SNE component 2')
plt.show()

In [ ]:
from sklearn.manifold import TSNE

# Effectuer une réduction de dimensionnalité en utilisant UMAP avec 3 composantes
umap = UMAP(n_components=3, random_state=42)
data_umap = umap.fit_transform(images_np)

# Crée un DataFrame avec les données réduites
umap_df = pd.DataFrame(data_umap, columns=['Component 1', 'Component 2', 'Component 3'])

# Trace les données réduites en utilisant un graphique 3D
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

for i in range(12):  # Supposant que les classes sont étiquetées de 0 à 1.
    mask = labels_np == i
    ax.scatter(umap_df['Component 1'][mask], umap_df['Component 2'][mask], umap_df['Component 3'][mask], label=f'{image[i]}', s = 10)

ax.legend()
ax.set_xlabel('UMAP component 1')
ax.set_ylabel('UMAP component 2')
ax.set_zlabel('UMAP component 3')
plt.show()


# Predictions

In [11]:
from torch.utils.data import DataLoader, Dataset, random_split
# Adapter la taille des datasets
df_test = CustomDataset(image_paths_test, labels_test, transform=transformation)
test_dataset_size = len(df_test)
split_sizes = [test_dataset_size // 2, test_dataset_size - test_dataset_size // 2]
test_online, test_dataset = random_split(df_test, split_sizes)
test_online_indices = test_online.indices[:300]
test_online = torch.utils.data.Subset(df_test, test_online_indices)


online_loader = DataLoader(test_online, batch_size = 1024, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=1024, shuffle=True)

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18


# 1. Charger le modèle ResNet18 pré-entraîné
model = resnet18(pretrained=False)

# Adapter le modèle pour une sortie unique (classification binaire)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 2),
            # nn.ReLU(),
            # nn.Linear(512, 2)
        )

# 2. Définir le critère et l'optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(
                model.parameters(), lr=0.01, weight_decay=0.01
            )
sched = StepLR(optimizer, step_size=10, gamma=0.1)
# 3. Déplacer le modèle vers le GPU si disponible
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training phase using online_loader
    model.train()
    for i, (inputs, labels) in enumerate(online_loader, 0):
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    sched.step()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(online_loader)}")

print('Finished Training')

# 4. Testing on testloader
correct_test = 0
total_test = 0
with torch.no_grad():
    model.eval()
    for i, (inputs, labels) in enumerate(testloader, 0):
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        outputs = model(inputs)
        predicted_labels = torch.argmax(outputs, dim=1)
        labels_ = torch.argmax(labels, dim=1)
        correct_test += (predicted_labels == labels_).sum().item()

        total_test += labels.size(0)

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy}%")

Epoch 1, Loss: 0.7026586532592773
Epoch 2, Loss: 2.200211763381958
Epoch 3, Loss: 1.4535274505615234
Epoch 4, Loss: 0.8095605373382568
Epoch 5, Loss: 0.7049943208694458
Epoch 6, Loss: 0.6871241927146912
Epoch 7, Loss: 0.7027666568756104
Epoch 8, Loss: 0.6548987030982971
Epoch 9, Loss: 0.5824663639068604
Epoch 10, Loss: 0.5156244039535522
Epoch 11, Loss: 0.41505053639411926
Epoch 12, Loss: 0.3957947790622711
Epoch 13, Loss: 0.3861728608608246
Epoch 14, Loss: 0.37021663784980774
Epoch 15, Loss: 0.3427679240703583
Epoch 16, Loss: 0.3300860822200775
Epoch 17, Loss: 0.3128366768360138
Epoch 18, Loss: 0.29459574818611145
Epoch 19, Loss: 0.28798574209213257
Epoch 20, Loss: 0.26294928789138794
Epoch 21, Loss: 0.25266551971435547
Epoch 22, Loss: 0.24384705722332
Epoch 23, Loss: 0.24188119173049927
Epoch 24, Loss: 0.23794806003570557
Epoch 25, Loss: 0.23206642270088196
Epoch 26, Loss: 0.2295873761177063
Epoch 27, Loss: 0.22767110168933868
Epoch 28, Loss: 0.2164156436920166
Epoch 29, Loss: 0.2131